### Ejercicio 4

Para realizar nuestra simulacion de Simpy, definimos nuestro escenario Banco, al cual le asignamos un recurso de capacidad 1, es decir, que sólo una persona a la vez puede estar utilizando nuestro recurso ATM a la vez. Este recurso se corresponde con un elemento Resource de Simpy, y al mismo le otorgamos una capacidad de 1. 

Para la generacion de la llegada de personas, aplicamos una distribución exponencial, y por cada llegada, creamos una nueva Persona, que intenta utilizar el recurso(nuestra ATM).

El tiempo de utilización del recurso se basa en una distribución uniforme, con probabilidades dependiendo del tipo de persona que llega. Además, consideramos un grado de tolerancia, que también se basa en una distribución uniforme.


Finalmente tenemos dos atributos en nuestro escenario, donde guardamos la cantidad máxima de personas en la fila, y otro para registrar a la persona que tardo mas tiempo para utilizar el cajero.

In [28]:
import simpy
import numpy as np
import random

class Banco:
    maxPersonasEnFila = 0
    maxTiempoEnFila = 0
    
    def __init__(self):
        self.maxPersonasEnFila = 0
        self.maxTiempoEnFila = 0  

    def banco(self,env):
        
        atm = simpy.Resource(env,capacity=1)
        cantMaxPersonasEnFila = 0  
        print("Esperando que abra el banco...")
        yield env.timeout(10*3600)
        print("----------------Abrimos!----------------------")
        personasList = list()
        i = 0;
        while True:  
           #Calculo la llegada de las personas
            t = random.expovariate(frecuenciaArribo(env))
            yield env.timeout(t)

            persona = Persona() 
            i+=1;
            persona.nombre = str(i);
            print(getTimeFromSeconds(env.now), "Llega al cajero ", persona.tipo + "-" + persona.nombre)
            env.process(self.cajero(env, atm, persona))

    
    def cajero(self,env,atm,persona):
            with atm.request() as req:
                print(getTimeFromSeconds(env.now),"--Cantidad de personas en fila: ", len(atm.queue))
                if (self.maxPersonasEnFila < len(atm.queue)):
                    self.maxPersonasEnFila = len(atm.queue)

                #Seteamos el tiempo de espera de la persona.
                persona.tiempoEspera = env.now
                yield req

                tiempoEspera = (env.now - persona.tiempoEspera);
                if (tiempoEspera > 0):
                    print("---Usando finalmente ATM--: ", persona.nombre)
                    print("Tiempo de espera en fila : ", getMinutesFromSeconds(tiempoEspera))
                
                if (self.maxTiempoEnFila < tiempoEspera):
                    self.maxTiempoEnFila = tiempoEspera
                          
                tiempoIngreso = env.now
                yield env.timeout(persona.tiempoCajero*60)
                          
                print(getTimeFromSeconds(env.now), "-- Sale del cajero: ", persona.tipo + "-" + persona.nombre)
                print("---Tiempo de uso --: ", getMinutesFromSeconds(env.now - tiempoIngreso))
    

    def getTimeFromSeconds(seconds):
        m, s = divmod(seconds, 60)
        h, m = divmod(m, 60)
        return "%d:%02d:%02d" % (h, m, s)

    def getMinutesFromSeconds(seconds):
        m, s = divmod(seconds, 60)
        return "%02d.%02d" % (m, s) + " Minutos"


    def frecuenciaArribo(env):
        if (env.now <= 12*3600):
            return 1.0/260.0
        if (env.now <= 15*3600):
            return 1.0/120.0
        return 1.0/360.0
    
    def getMaxTiempo(self):
        return (getMinutesFromSeconds(self.maxTiempoEnFila))

class Persona:
    tipo = ""
    tiempoCajero = 1
    nombre = ""
    tiempoEspera = 1
    
    def __init__(self, tipo,tiempoCajero, tiempoEspera):
        self.tipo = tipo
        self.tiempoCajero = tiempoCajero
        self.tiempoEspera = tiempoEspera
        
    def __init__(self):
        val = random.uniform(0,1);
        if (val <= 0.1):
            #print("--Persona tipo 1--")
            self.tipo = "Tipo 1" #Tolerancia +-3
            if (random.uniform(0,1) >= .5):
                self.tiempoCajero = 4.0 + (np.random.uniform(0,3));
            else:
                self.tiempoCajero = 4.0 + (np.random.uniform(0,3))*-1;
        elif (val <= .8):
            #print("--Persona tipo 2--")
            self.tipo = "Tipo 2" #Tolerancia +-1
            if (random.uniform(0,1) >= .5):
                self.tiempoCajero = 2.0 + np.random.uniform(0,1); 
            else:    
                self.tiempoCajero = 2.0 + np.random.uniform(0,1)*-1; 

        #print("--Persona tipo 3--")
        else:
            self.tipo = "Tipo 3" #Tolerancia 3+-2
            if (random.uniform(0,1) >= .5):
                self.tiempoCajero = 3.0 + np.random.uniform(0,2); 
            else:    
                self.tiempoCajero = 3.0 + np.random.uniform(0,2)*-1;
                
                
                
env = simpy.Environment()

banco = Banco()
env.process(banco.banco(env))
env.run(until=19*3600)

print("--------------------Banco Cerrado :( ----------------------------")
print('\r\n')
print(" -----------------------RESULTADOS-------------------------------")

print(" CANTIDAD MAXIMA DE PERSONAS EN FILA: ",banco.maxPersonasEnFila)
print(" TIEMPO DE MAXIMA DE PERSONAS: ",banco.getMaxTiempo())

Esperando que abra el banco...
----------------Abrimos!----------------------
('10:07:55', 'Llega al cajero ', 'Tipo 2-1')
('10:07:55', '--Cantidad de personas en fila: ', 0)
('10:09:24', '-- Sale del cajero: ', 'Tipo 2-1')
('---Tiempo de uso --: ', '01.29 Minutos')
('10:14:12', 'Llega al cajero ', 'Tipo 2-2')
('10:14:12', '--Cantidad de personas en fila: ', 0)
('10:15:19', '-- Sale del cajero: ', 'Tipo 2-2')
('---Tiempo de uso --: ', '01.07 Minutos')
('10:19:22', 'Llega al cajero ', 'Tipo 2-3')
('10:19:22', '--Cantidad de personas en fila: ', 0)
('10:20:16', 'Llega al cajero ', 'Tipo 2-4')
('10:20:16', '--Cantidad de personas en fila: ', 1)
('10:20:59', 'Llega al cajero ', 'Tipo 2-5')
('10:20:59', '--Cantidad de personas en fila: ', 2)
('10:22:06', '-- Sale del cajero: ', 'Tipo 2-3')
('---Tiempo de uso --: ', '02.44 Minutos')
('---Usando finalmente ATM--: ', '4')
('Tiempo de espera en fila : ', '01.50 Minutos')
('10:23:03', 'Llega al cajero ', 'Tipo 2-6')
('10:23:03', '--Cantidad de p